In [83]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
pd.options.mode.use_inf_as_na = True
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix
import time
from sklearn import tree

In [28]:
#https://www.kaggle.com/kaggle/kaggle-survey-2018#multipleChoiceResponses.csv

In [39]:
start_time = time.time()
df = pd.read_csv('multipleChoiceResponses.csv')
display(df.head())
print("--- %s seconds ---" % (time.time() - start_time))

c:\users\greg\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (157,172,174,210,218,219,246,368,371,384,389,390,391,393) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,...,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
0,710,Female,-1,45-49,United States of America,Doctoral degree,Other,Consultant,-1,Other,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,434,Male,-1,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,0,Manufacturing/Fabrication,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,718,Female,-1,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,-1,I am a student,...,-1,NaN,Too time-consuming,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,621,Male,-1,35-39,United States of America,Master’s degree,"Social sciences (anthropology, psychology, soc...",Not employed,-1,NaN,...,-1,NaN,NaN,Requires too much technical knowledge,NaN,Not enough incentives to share my work,NaN,NaN,NaN,-1
4,731,Male,-1,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,-1,I am a student,...,-1,NaN,Too time-consuming,NaN,NaN,Not enough incentives to share my work,NaN,NaN,NaN,-1


--- 1.5326282978057861 seconds ---


In [41]:
#drop all columns with mixed datatypes
df = df.drop(df.columns[[157,172,174,210,218,219,246,368,371,384,389,390,391,393]], axis=1)
df.groupby('Q1').agg('count')

,Time from Start to Finish (seconds),Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,Q7_OTHER_TEXT,...,Q49_Part_8,Q49_Part_9,Q49_Part_10,Q49_Part_11,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_7,Q50_OTHER_TEXT
Q1,,,,,,,,,,,,,,,,,,,,,
Female,4010,4010,4010,4010,3943,3880,3851,4010,3598,4010,...,624,498,663,167,4010,150,890,422,413,4010
Male,19430,19430,19430,19430,19093,18693,18668,19430,17748,19430,...,3910,3097,3632,803,19430,1191,5495,2225,2042,19430
Prefer not to say,340,340,340,340,324,299,303,340,267,340,...,57,46,50,13,340,22,65,23,37,340
Prefer to self-describe,79,79,79,79,78,75,78,79,72,79,...,23,24,21,2,79,8,28,4,8,79


In [63]:

Y = df['Q1'].astype(str)


# Convert to dummies

In [62]:
#Function to truncate dummies to a manageable amount
def dum_sign(dummy_col, threshold=0.1):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col) / len(dummy_col)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)


In [78]:
cols_to_dummy = ['Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10']

X = pd.DataFrame(df.index, columns=['index'])
for col in cols_to_dummy:
    dums = dum_sign(df[col]).reset_index()
    X = X.merge(dums, on='index')
X.head()

,index,Q2_18-21,Q2_22-24,Q2_25-29,Q2_30-34,Q2_others,Q3_India,Q3_United States of America,Q3_others,Q4_Bachelor’s degree,...,Q8_others,"Q9_0-10,000",Q9_I do not wish to disclose my approximate yearly compensation,Q9_others,Q10_I do not know,Q10_No (we do not use ML methods),Q10_We are exploring ML methods (and may one day put a model into production),"Q10_We have well established ML methods (i.e., models in production for more than 2 years)","Q10_We recently started using ML methods (i.e., models in production for less than 2 years)",Q10_others
0,0,0,0,0,0,1,0,1,0,0,...,1,0,0,1,1,0,0,0,0,0
1,1,0,0,0,1,0,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
3,3,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1
4,4,0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0


# TREE

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

start_time = time.time()
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_features=1)

dtree.fit(X_train, y_train)
display(dtree.score(X_test, y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.7000838222967309

--- 0.14319324493408203 seconds ---


# Simplest Forest

In [88]:
start_time = time.time()
forest = ensemble.RandomForestClassifier(n_estimators=5)

forest.fit(X_train, y_train)
display(forest.score(X_test, y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.7549036043587595

--- 0.3879115581512451 seconds ---
